<a href="https://colab.research.google.com/github/martin-fabbri/colab-notebooks/blob/master/tf-serving/tf_serving_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Train and serve a TensorFlow model with TensorFlow Serving

In [1]:
%%capture
# !pip install -U grpcio~=1.32.0 ?? Does it come with TF?

In [18]:
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Conv2D

import numpy as np
import matplotlib.pyplot as plt
import os
import subprocess

print("tensorflow:", tf.__version__)

tensorflow: 2.4.1


In [6]:
!nvidia-smi

Wed Mar  3 06:00:26 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Create the model

In [15]:
(train_images, train_labels), (
    test_images,
    test_labels,
) = fashion_mnist.load_data()
train_images = train_images / 255.0
test_images = test_images / 255.0
num_images, width, height = train_images.shape
train_images = np.expand_dims(train_images, axis=3)
test_images = np.expand_dims(test_images, axis=3)

class_names = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]
print(f"\ntrain_images.shape: {train_images.shape}, of {train_images.dtype}")
print(f"test_images.shape: {test_images.shape}, of {test_images.dtype}")


train_images.shape: (60000, 28, 28, 1), of float64
test_images.shape: (10000, 28, 28, 1), of float64


# Train and evaluate your model

In [20]:
model = Sequential([
    Conv2D(input_shape=(28, 28, 1), filters=8, kernel_size=3, strides=2, activation="relu", name="Conv1"),
    Flatten(),
    Dense(10, activation=tf.nn.softmax, name="Softmax")
])
model.summary()

testing = False
epochs = 5

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)
model.fit(train_images, train_labels, epochs=epochs)

test_loss, test_acc = model.evaluate(test_images, test_labels)
print("test accuracy:", test_acc)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv1 (Conv2D)               (None, 13, 13, 8)         80        
_________________________________________________________________
flatten_1 (Flatten)          (None, 1352)              0         
_________________________________________________________________
Softmax (Dense)              (None, 10)                13530     
Total params: 13,610
Trainable params: 13,610
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
1875/1875 [==============================] - 19s 2ms/step - loss: 0.7268 - accuracy: 0.7483
Epoch 2/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.4220 - accuracy: 0.8506
Epoch 3/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3857 - accuracy: 0.8618
Epoch 4/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0